In [38]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import cv2
from torchvision import transforms
from torch.optim import SGD
from torch.utils.data import random_split, DataLoader
from imutils import paths
import os
import torch.nn.functional as F


In [39]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x


In [40]:
def normalizeAndProcessing(img: np.ndarray):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize((28, 28)),
        transforms.Normalize((0.5,), (0.5,))
    ])
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    processed_img = transform(gray_img)
    processed_img = processed_img.unsqueeze(0)  # Add batch dimension
    return processed_img.to(device)


In [41]:
# Initialize the CNN model
model = SimpleCNN().to(device)

# Example inference and loss calculation
img = cv2.imread(list_dir_test[10])
new_img = normalizeAndProcessing(img)

criterion = nn.BCELoss()
res = model(new_img)
print(res)

target = torch.tensor([[0.0]]).to(device)
loss = criterion(res, target)


tensor([[0.5029]], device='cuda:0', grad_fn=<SigmoidBackward0>)


In [42]:
max_epochs = 300
train_losses, test_losses, test_accuracy = [], [], []

for epoch in range(max_epochs):
    model.train()
    train_loss = 0.0
    total = 0.0
    correct = 0.0
    for img, label in train_loader:
        img, label = img.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(img)
        loss = criterion(output, label)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * img.size(0)
        _, predicted = torch.max(output.data, 1)
        total += label.size(0)
        correct += (predicted == label).sum().item()
    train_losses.append(train_loss / len(train_loader.dataset))
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{max_epochs}, Train Loss: {train_losses[-1]}, Train Accuracy: {train_accuracy}%")

    model.eval()
    test_loss = 0.0
    correct = 0.0
    total = 0.0
    with torch.no_grad():
        for img, label in test_loader:
            img, label = img.to(device), label.to(device)
            output = model(img)
            loss = criterion(output, label)
            test_loss += loss.item() * img.size(0)
            _, predicted = torch.max(output.data, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
    test_losses.append(test_loss / len(test_loader.dataset))
    test_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{max_epochs}, Test Loss: {test_losses[-1]}, Test Accuracy: {test_accuracy}%")


NameError: name 'train_loader' is not defined